In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
from datetime import datetime

import xgboost as xgb

In [2]:
train=pd.read_csv('/content/drive/MyDrive/colab전용/train.csv')
test=pd.read_csv('/content/drive/MyDrive/colab전용/test.csv')
submission=pd.read_csv('/content/drive/MyDrive/colab전용/submission.csv')

display(train.head())

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


In [3]:
# train 과 test 가 관련이 있다는 가정.
# 하지만, train과 test의 store_id 가 숫자는 같아도 다를 수 있다는 사실 때문에 다른 변수와의 관계가 크지 않을것이라고 생각한다.
# 그래서 일변량 시계열 예측문제로 생각하기로 했다. 

train=train.drop(['time','card_id','installments','days_of_week','holyday'],axis=1)
test=test.drop(['time','card_id','installments','days_of_week','holyday'],axis=1)

display(train.head())

,store_id,date,amount
0,0,2016-12-14,5
1,0,2016-12-14,-5
2,0,2016-12-19,144
3,0,2016-12-19,66
4,0,2016-12-19,24


In [4]:
train=train.groupby(['date','store_id'],as_index=False)['amount'].sum()
test=test.groupby(['date','store_id'],as_index=False)['amount'].sum()

display(train.head())

,date,store_id,amount
0,2016-08-01,2,2372
1,2016-08-01,6,1080
2,2016-08-01,8,3650
3,2016-08-01,12,1155
4,2016-08-01,14,1270


In [5]:
train['date']=train['date'].apply(pd.to_datetime)
test['date']=test['date'].apply(pd.to_datetime)

train['temp_date']=train['date']
test['temp_date']=test['date']

train=train.set_index('date')
test=test.set_index('date')

display(train.head())

,store_id,amount,temp_date
date,,,
2016-08-01,2,2372,2016-08-01
2016-08-01,6,1080,2016-08-01
2016-08-01,8,3650,2016-08-01
2016-08-01,12,1155,2016-08-01
2016-08-01,14,1270,2016-08-01


In [6]:
# train 할 최소 양.
limit=160

print(f"before train shape {train.shape}")
for store in train.store_id.unique():
    if train[train.store_id==store].amount.count()<limit:
        train=train[train.store_id!=store]

print(f"after train shape {train.shape}")

before train shape (480160, 3)
after train shape (432671, 3)


In [7]:
# 기존 2등 코드와 다른점이 있다면 
# test는 신경쓰지 않았다. 있는 그대로 가져가는것이 맞다고 생각했다.
# 또, train test 모두 매출이 없을때가 많다. 마지막날 7월 31일 기준으로 7일전까지 없으면 그 상점은 버리기로 했다. 
# test[test.store_id==183].asfreq('d',fill_value=0).tail(30)
# 실제로 test의 183번 상점 같은 경우처럼 재수없게 마지막날짜로부터 7일의 매출이 없을 수도 있다.

t2=datetime.strptime('2018-07-31 00:00:00','%Y-%m-%d %H:%M:%S')

print(f"before train shape {train.shape}")
for store in train.store_id.unique():
    df=train[train.store_id==store]
    if str(df.iloc[-1].temp_date)!='2018-07-31 00:00:00':
        t1=datetime.strptime(str(df.iloc[-1].temp_date),'%Y-%m-%d %H:%M:%S')
        diff=t2-t1
        if diff.days>8:
            train=train[train.store_id!=store]

print(f"after train shape {train.shape}")

before train shape (432671, 3)
after train shape (406986, 3)


In [8]:
# 2등의 코드를 분석했을 때, 어차피 store_id 에 따른 card_id부터 installments까지 버렸기 때문에
# store_id의 시계열 특성만 파생변수로 만들었다. 

def reform_data(df,is_train):
    arr=[]

    for num in df.store_id.unique():
        store=df[df.store_id==num]

        store=store.asfreq('d',fill_value=0)
        store['temp_date']=store.index
        store['store_id']=num

        # 이동평균 파생변수
        store['MA7']=store['amount'].rolling('7d').mean()
        store['MA15']=store['amount'].rolling('15d').mean()
        store['MA30']=store['amount'].rolling('30d').mean()

        if is_train:
            store_y=store.last('100d')
            target=store_y.amount.sum()
            store_x=store[store.temp_date<store_y.iloc[0].temp_date]
        else:
            target=0
            store_x=store[:]
        
        stacked=[]
        stacked.append(num) # store_id
        stacked.append(target) # target
        stacked.append(store_x.amount.mean()) # mean of amount
        stacked.append(store_x.amount.median()) # median of amount
        
        stacked.append(store_x.last('7d').amount.mean()) # mean of amount (last 7 days)
        stacked.append(store_x.last('15d').amount.mean()) # mean of amount (last 15 days)
        stacked.append(store_x.last('30d').amount.mean()) # mean of amount (last 30 days)
        
        stacked.append(store_x.last('7d').amount.median()) # median of amount (last 7 days)
        stacked.append(store_x.last('15d').amount.median()) # median of amount (last 15 days)
        stacked.append(store_x.last('30d').amount.median()) # median of amount (last 30 days)
        
        stacked.append(store_x.last('7d').amount.sum()) # sum of amount (last 7 days)
        stacked.append(store_x.last('15d').amount.sum()) # sum of amount (last 15 days)
        stacked.append(store_x.last('30d').amount.sum()) # sum of amount (last 30 days)
        
        stacked.append(store_x.last('7d').MA7.mean()) # mean of Moving Average of 7D (last 7 days)
        stacked.append(store_x.last('15d').MA7.mean()) # mean of Moving Average of 7D (last 15 days)
        stacked.append(store_x.last('30d').MA7.mean()) # mean of Moving Average of 7D (last 30 days)
        stacked.append(store_x.last('7d').MA15.mean()) # mean of Moving Average of 15D (last 7 days)
        stacked.append(store_x.last('15d').MA15.mean()) # mean of Moving Average of 15D (last 15 days)
        stacked.append(store_x.last('30d').MA15.mean()) # mean of Moving Average of 15D (last 30 days)
        stacked.append(store_x.last('7d').MA30.mean()) # mean of Moving Average of 30D (last 7 days)
        stacked.append(store_x.last('15d').MA30.mean()) # mean of Moving Average of 30D (last 15 days)
        stacked.append(store_x.last('30d').MA30.mean()) # mean of Moving Average of 30D (last 30 days)
        
        arr.append(stacked) # Append the 'stacked' array in to 'x_array'
        
    return arr

train=reform_data(train,True)
test=reform_data(test,True)

In [9]:
# 60일의 데이터를 가지고(60일->100일전 의 feature) 100일을 예측하는 (100일 후의 y)
train = pd.DataFrame(train, columns=['store_id', 'target', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])
test = pd.DataFrame(test, columns=['store_id', 'target', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])

display(train)
display(test)

,store_id,target,mean,median,7mean,15mean,30mean,7median,15median,30median,7sum,15sum,30sum,7ma7mean,15ma7mean,30ma7mean,7ma15mean,15ma15mean,30ma15mean,7ma30mean,15ma30mean,30ma30mean
0,2,186853,2266.995238,2267.0,2207.714286,2209.466667,2272.233333,2174.0,2174.0,2293.0,15454,33142,68167,2201.020408,2203.819048,2297.928571,2203.819048,2233.044444,2319.146667,2297.928571,2319.146667,2344.041111
1,6,219761,1857.495238,1080.0,2076.428571,2359.133333,1970.233333,1670.0,1670.0,1622.5,14535,35387,59107,2703.020408,2448.704762,2690.871429,2448.704762,2061.066667,2833.988889,2690.871429,2833.988889,3410.305556
2,8,79695,691.809524,500.0,623.571429,704.333333,650.666667,260.0,450.0,475.0,4365,10565,19520,570.102041,659.047619,591.166667,659.047619,635.622222,629.444444,591.166667,629.444444,664.450000
3,12,147744,1224.731746,898.5,1995.000000,1524.333333,1118.200000,2145.0,1155.0,990.0,13965,22865,33546,1772.346939,1273.571429,971.723810,1273.571429,990.880000,860.555556,971.723810,860.555556,824.680000
4,14,17253,309.682035,150.0,267.142857,218.733333,340.766667,195.0,187.0,153.5,1870,3281,10223,252.836735,471.485714,341.904762,471.485714,465.400000,346.037778,341.904762,346.037778,305.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,242,55006,512.181818,395.0,758.714286,580.733333,608.233333,530.0,505.0,454.5,5311,8711,18247,421.469388,537.009524,530.876190,537.009524,579.191111,554.102222,530.876190,554.102222,544.604444
922,120,176547,1418.546667,1238.0,1524.000000,1503.066667,1360.633333,1586.0,1311.0,1109.5,10668,22546,40819,1416.367347,1414.380952,1278.057143,1414.380952,1344.075556,1251.404444,1278.057143,1251.404444,1253.010000
923,1782,101682,1727.285714,1625.0,1644.285714,1474.000000,1714.333333,1680.0,1505.0,1595.0,11510,22110,51430,1531.428571,1402.190476,1753.809524,1402.190476,1482.822222,1837.511111,1753.809524,1837.511111,1986.222222
924,904,62155,332.205882,287.5,577.857143,506.000000,470.333333,560.0,510.0,425.0,4045,7590,14110,509.183673,477.523810,425.833333,477.523810,454.933333,407.855556,425.833333,407.855556,361.844444


,store_id,target,mean,median,7mean,15mean,30mean,7median,15median,30median,7sum,15sum,30sum,7ma7mean,15ma7mean,30ma7mean,7ma15mean,15ma15mean,30ma15mean,7ma30mean,15ma30mean,30ma30mean
0,0,110729,1038.440945,968.0,1193.000000,1009.200000,1001.800000,1256.0,1080.0,878.5,8351,15138,30054,1077.142857,1001.657143,991.661905,1001.657143,979.426667,943.368889,991.661905,943.368889,894.794444
1,2,36160,497.142012,170.0,153.571429,643.000000,490.166667,0.0,0.0,232.5,1075,9645,14705,523.571429,637.380952,484.190476,637.380952,579.933333,622.133333,484.190476,622.133333,664.372222
2,3,43130,366.444882,324.5,234.428571,309.200000,322.833333,122.0,302.0,310.0,1641,4638,9685,248.428571,333.428571,346.133333,333.428571,358.888889,363.102222,346.133333,363.102222,359.690000
3,4,20532,271.732283,246.5,160.714286,226.800000,237.633333,211.0,254.0,238.0,1125,3402,7129,182.061224,231.266667,232.390476,231.266667,240.848889,227.324444,232.390476,227.324444,230.455556
4,6,81838,951.797244,903.0,689.000000,709.266667,725.533333,463.0,649.0,694.5,4823,10639,21766,623.346939,645.552381,719.928571,645.552381,687.866667,756.906667,719.928571,756.906667,835.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,84,11192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,61,55053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,192,88257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,32,61415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
cols = [col for col in train.columns if col not in ['store_id','target']]
y='target'

X_train=train[cols]
y_train=train[y]
X_test=test[cols]

In [11]:
# 다시 못찾았는데 시계열 가격 같이 어떠한 연속적인 숫자를 예측할때에는 회귀 objective 와 rmse,mse,mae 등 직접적인 metric이 좋다고 했다.

dtrain=xgb.DMatrix(X_train,label=y_train)
params={
    'objective':'reg:squarederror',
    'eval_metric':'mae',
    'seed':71
}
history=xgb.cv(params,dtrain,num_boost_round=500,nfold=4,seed=71)

In [12]:
# 평균이 16000인데 떨어진 정도가 1/4 이면 좀 많이 튄거같다. 
history.tail()

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
495,0.064226,0.013468,16876.739502,4340.447762
496,0.063633,0.013460,16876.740478,4340.447211
497,0.063115,0.013377,16876.740235,4340.447441
498,0.062627,0.013390,16876.740478,4340.447211
499,0.062410,0.013277,16876.740478,4340.447211


In [13]:
# 파라미터를 조정한다.

dtrain=xgb.DMatrix(X_train,label=y_train)
params={
    'objective':'reg:squarederror',
    'booster':'gblinear',
    'eta':0.05,
    'max_depth':10,
    'colsample_bytree':0.7,
    'alpha':0.01,
    'eval_metric':'mae',
    'seed':71
}
history=xgb.cv(params,dtrain,num_boost_round=500,nfold=4,seed=71)

In [14]:
# 이정도면 해도 될것 같다는 생각을 했다. 
history.tail() 

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
495,12415.582031,334.712460,12925.936035,1096.583517
496,12415.540772,334.740176,12926.044433,1096.627203
497,12415.499267,334.768949,12926.153076,1096.670795
498,12415.455810,334.798163,12926.269287,1096.715703
499,12415.418457,334.829262,12926.365479,1096.760522


In [15]:
# https://light-tree.tistory.com/125
# 정규화에 대한 이해가 아직 덜 하지만, outlier의 영향을 덜 받게. 
# 그리고 실제값과 예측값의 차이를 예측하는것이 모델의 overfitting도 낮추는 좋은 방법이라고 생각했다.
dtrain=xgb.DMatrix(X_train,label=y_train)

params={
    'objective':'reg:squarederror',
    'booster':'gblinear',
    'eta':0.05,
    'max_depth':10,
    'colsample_bytree':0.7,
    'alpha':0.01,
    'eval_metric':'mae',
    'seed':71
}

model=xgb.train(params,dtrain,num_boost_round=500)

In [16]:
# gblinear 에는 변수 중요도가 없다.

# fig, ax = plt.subplots(figsize=(12,18))
# xgb.plot_importance(model, max_num_features=10, ax=ax)
# plt.show()

In [17]:
# 마지막 2월과 3월의 비율 처리는 안했다. 그런건 상관 없기때문이다.
# 그리고 최적화 상수도 안곱했다. 그 뜻을 모르겠다. 

dtest=xgb.DMatrix(X_test)

pred=model.predict(dtest)
submission.total_sales=pred
display(submission.head())

,store_id,total_sales
0,0,98440.226562
1,1,49526.500000
2,2,35626.070312
3,3,26367.720703
4,4,78163.507812


In [20]:
ewm=pd.read_csv('/content/drive/MyDrive/colab전용/ewm7_2021-01-05.csv')
display(ewm.head())

,store_id,total_sales
0,0,87198.309390
1,1,40294.893984
2,2,32249.717307
3,3,45659.444895
4,4,24543.926970


In [23]:
submission.total_sales=(submission.total_sales+ewm.total_sales)/2

In [24]:
# submission.to_csv('/content/drive/MyDrive/colab전용/ensemble(xgb,ewm)submission.csv',index=False)